<a href="https://colab.research.google.com/github/shicong0720/public/blob/master/2_13_Query_OSM_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up environment

In [1]:
!pip install pyproj

In [2]:
import pandas as pd
from google.colab import drive
import os
import seaborn as sns
import numpy as np
import shapely
import math

In [3]:
drive.mount('/content/gdrive',force_remount=True)
os.chdir("/content/gdrive/My Drive/big data and city 2/")

Mounted at /content/gdrive


# Get 500 cities boundary 

In [4]:
% pip install geopandas

In [5]:
import geopandas as gpd


/usr/local/lib/python3.6/dist-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [6]:
area_boundary = gpd.read_file("500Cities_Tracts_Clip.geojson")
area_boundary.head()


,place2010,tract2010,ST,PlaceName,plctract10,PlcTrPop10,geometry
0,0107000,01073000100,01,Birmingham,0107000-01073000100,3042,"POLYGON ((-9653001.056 3974630.899, -9652982.0..."
1,0107000,01073000300,01,Birmingham,0107000-01073000300,2735,"POLYGON ((-9657462.853 3968794.996, -9657470.2..."
2,0107000,01073000400,01,Birmingham,0107000-01073000400,3338,"POLYGON ((-9657275.613 3972217.160, -9657275.3..."
3,0107000,01073000500,01,Birmingham,0107000-01073000500,2864,"POLYGON ((-9660666.407 3968887.964, -9660631.7..."
4,0107000,01073000700,01,Birmingham,0107000-01073000700,2577,"POLYGON ((-9661391.097 3968591.429, -9661398.2..."


In [7]:
area_boundary['area'] = np.nan
for i in range(len(area_boundary)):
  area_boundary.loc[i,'area'] = area_boundary.loc[i,'geometry'].area


In [8]:
area_boundary.head()


,place2010,tract2010,ST,PlaceName,plctract10,PlcTrPop10,geometry,area
0,0107000,01073000100,01,Birmingham,0107000-01073000100,3042,"POLYGON ((-9653001.056 3974630.899, -9652982.0...",1.102526e+07
1,0107000,01073000300,01,Birmingham,0107000-01073000300,2735,"POLYGON ((-9657462.853 3968794.996, -9657470.2...",3.020969e+06
2,0107000,01073000400,01,Birmingham,0107000-01073000400,3338,"POLYGON ((-9657275.613 3972217.160, -9657275.3...",1.155412e+07
3,0107000,01073000500,01,Birmingham,0107000-01073000500,2864,"POLYGON ((-9660666.407 3968887.964, -9660631.7...",6.955719e+06
4,0107000,01073000700,01,Birmingham,0107000-01073000700,2577,"POLYGON ((-9661391.097 3968591.429, -9661398.2...",5.082370e+06


# Get Health Data from CDC

In [9]:
health_data = pd.read_csv("500_Cities__Census_Tract-level_Data__GIS_Friendly_Format___2018_release.csv")

In [10]:
health_data.head()

,StateAbbr,PlaceName,PlaceFIPS,TractFIPS,Place_TractID,Population2010,ACCESS2_CrudePrev,ACCESS2_Crude95CI,ARTHRITIS_CrudePrev,ARTHRITIS_Crude95CI,BINGE_CrudePrev,BINGE_Crude95CI,BPHIGH_CrudePrev,BPHIGH_Crude95CI,BPMED_CrudePrev,BPMED_Crude95CI,CANCER_CrudePrev,CANCER_Crude95CI,CASTHMA_CrudePrev,CASTHMA_Crude95CI,CHD_CrudePrev,CHD_Crude95CI,CHECKUP_CrudePrev,CHECKUP_Crude95CI,CHOLSCREEN_CrudePrev,CHOLSCREEN_Crude95CI,COLON_SCREEN_CrudePrev,COLON_SCREEN_Crude95CI,COPD_CrudePrev,COPD_Crude95CI,COREM_CrudePrev,COREM_Crude95CI,COREW_CrudePrev,COREW_Crude95CI,CSMOKING_CrudePrev,CSMOKING_Crude95CI,DENTAL_CrudePrev,DENTAL_Crude95CI,DIABETES_CrudePrev,DIABETES_Crude95CI,HIGHCHOL_CrudePrev,HIGHCHOL_Crude95CI,KIDNEY_CrudePrev,KIDNEY_Crude95CI,LPA_CrudePrev,LPA_Crude95CI,MAMMOUSE_CrudePrev,MAMMOUSE_Crude95CI,MHLTH_CrudePrev,MHLTH_Crude95CI,OBESITY_CrudePrev,OBESITY_Crude95CI,PAPTEST_CrudePrev,PAPTEST_Crude95CI,PHLTH_CrudePrev,PHLTH_Crude95CI,SLEEP_CrudePrev,SLEEP_Crude95CI,STROKE_CrudePrev,STROKE_Crude95CI,TEETHLOST_CrudePrev,TEETHLOST_Crude95CI,Geolocation
0,CA,Los Angeles,644000,6037206032,0644000-06037206032,5275,28.0,"(22.7, 33.2)",19.9,"(18.8, 20.9)",13.0,"(12.3, 13.7)",31.7,"(30.6, 32.9)",70.0,"(68.6, 71.4)",4.5,"( 4.2, 4.7)",8.5,"( 7.7, 9.3)",6.9,"( 6.2, 7.6)",63.8,"(62.5, 65.1)",66.0,"(63.0, 69.0)",48.6,"(44.5, 53.0)",6.5,"( 5.6, 7.3)",18.2,"(14.5, 22.6)",22.1,"(18.4, 26.5)",15.6,"(12.8, 18.1)",43.6,"(38.7, 49.0)",15.5,"(14.4, 16.4)",38.3,"(36.8, 39.9)",4.1,"( 3.8, 4.4)",33.3,"(29.7, 36.5)",77.6,"(74.5, 80.5)",15.0,"(13.1, 16.7)",29.5,"(28.1, 30.8)",83.7,"(81.5, 86.0)",17.3,"(15.3, 19.2)",38.9,"(37.2, 40.4)",4.0,"( 3.5, 4.5)",18.8,"(12.6, 25.6)",POINT (-118.224698433 34.0470512474)
1,CA,Pasadena,656000,6037462001,0656000-06037462001,3974,23.3,"(19.2, 27.8)",16.7,"(16.0, 17.5)",16.3,"(15.6, 16.9)",27.1,"(26.1, 28.0)",63.7,"(62.4, 65.0)",3.6,"( 3.4, 3.7)",9.4,"( 8.8, 10.1)",4.8,"( 4.5, 5.2)",62.2,"(61.0, 63.3)",65.7,"(62.8, 68.4)",52.5,"(49.0, 56.0)",6.0,"( 5.3, 6.6)",19.4,"(16.7, 22.9)",25.7,"(22.2, 29.5)",17.8,"(15.6, 20.0)",47.6,"(43.4, 51.9)",11.6,"(10.9, 12.2)",31.8,"(30.6, 33.1)",3.2,"( 3.0, 3.4)",27.0,"(24.4, 29.6)",80.2,"(77.4, 82.7)",15.7,"(14.2, 17.3)",32.0,"(30.8, 33.1)",85.8,"(84.1, 87.5)",15.3,"(13.9, 16.8)",40.2,"(38.8, 41.5)",3.2,"( 2.9, 3.5)",18.1,"(13.7, 22.8)",POINT (-118.143832177 34.1633689905)
2,CA,Fullerton,628000,6059011504,0628000-06059011504,5473,11.7,"( 9.6, 14.5)",8.3,"( 7.9, 8.7)",18.6,"(17.8, 19.5)",13.6,"(13.0, 14.2)",49.5,"(48.0, 51.0)",2.0,"( 2.0, 2.1)",9.2,"( 8.6, 10.1)",2.1,"( 2.0, 2.3)",59.3,"(58.1, 60.5)",54.5,"(51.0, 57.4)",59.5,"(56.9, 62.2)",3.4,"( 3.0, 3.9)",34.6,"(29.8, 38.9)",27.5,"(23.9, 31.1)",13.3,"(11.1, 15.8)",61.3,"(57.0, 65.4)",4.6,"( 4.4, 4.9)",21.1,"(20.2, 22.2)",1.6,"( 1.5, 1.7)",16.1,"(14.2, 18.4)",81.0,"(78.9, 82.9)",14.1,"(12.4, 16.1)",18.7,"(17.6, 19.8)",78.2,"(75.4, 80.9)",8.5,"( 7.7, 9.5)",33.4,"(31.6, 35.1)",1.2,"( 1.1, 1.3)",11.9,"( 8.8, 16.0)",POINT (-117.883112998 33.8820402343)
3,TX,Longview,4843888,48183000401,4843888-48183000401,3371,19.3,"(16.8, 21.9)",21.1,"(20.2, 21.9)",20.9,"(20.1, 21.8)",28.4,"(27.4, 29.3)",72.4,"(71.3, 73.5)",5.2,"( 5.0, 5.5)",9.3,"( 8.8, 9.9)",5.4,"( 5.0, 5.8)",66.6,"(65.5, 67.6)",68.9,"(66.7, 71.0)",60.7,"(56.9, 64.2)",6.1,"( 5.5, 6.7)",35.6,"(28.9, 43.1)",30.5,"(25.3, 36.0)",20.5,"(18.0, 22.7)",58.0,"(54.8, 61.3)",9.2,"( 8.7, 9.7)",33.9,"(32.8, 35.1)",2.5,"( 2.4, 2.7)",25.1,"(23.0, 27.2)",74.3,"(70.7, 77.7)",13.5,"(12.4, 14.7)",33.9,"(32.4, 35.2)",79.3,"(76.5, 81.9)",11.8,"(10.8, 12.8)",34.9,"(33.4, 36.3)",2.7,"( 2.5, 3.0)",15.1,"(10.8, 20.4)",POINT (-94.7523262965 32.5544549842)
4,WI,Madison,5548000,55025002200,5548000-55025002200,4254,10.5,"( 8.7, 12.6)",23.3,"(21.9, 24.6)",25.0,"(24.2, 25.8)",27.9,"(26.6, 29.2)",74.6,"(73.5, 75.8)",6.8,"( 6.5, 7.1)",9.2,"( 8.7, 10.0)",6.0,"( 5.3, 6.7)",70.0,"(69.0, 70.9)",76.0,"(73.8, 78.0)",70.8,"(66.5, 74.6)",5.7,"( 4.8, 6.7)",29.7,"(23.7, 36.3)",31.6,"(26.1, 37.4)",16.6,"(14.0, 19.4)",72.1,"(68.7, 75.5)"

In [11]:
health_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27210 entries, 0 to 27209
Data columns (total 63 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   StateAbbr               27210 non-null  object 
 1   PlaceName               27210 non-null  object 
 2   PlaceFIPS               27210 non-null  int64  
 3   TractFIPS               27210 non-null  int64  
 4   Place_TractID           27210 non-null  object 
 5   Population2010          27210 non-null  int64  
 6   ACCESS2_CrudePrev       27209 non-null  float64
 7   ACCESS2_Crude95CI       27209 non-null  object 
 8   ARTHRITIS_CrudePrev     27210 non-null  float64
 9   ARTHRITIS_Crude95CI     27210 non-null  object 
 10  BINGE_CrudePrev         27210 non-null  float64
 11  BINGE_Crude95CI         27210 non-null  object 
 12  BPHIGH_CrudePrev        27210 non-null  float64
 13  BPHIGH_Crude95CI        27210 non-null  object 
 14  BPMED_CrudePrev         27210 non-null

# Process Health Data and get the Longitude and Latitude


In [12]:
health_data = health_data[['Geolocation','Place_TractID','Population2010','ACCESS2_CrudePrev','ARTHRITIS_CrudePrev','BINGE_CrudePrev','BPHIGH_CrudePrev','BPMED_CrudePrev','CANCER_CrudePrev','CASTHMA_CrudePrev', 'CHD_CrudePrev', 'CHECKUP_CrudePrev','CHOLSCREEN_CrudePrev','COLON_SCREEN_CrudePrev','COPD_CrudePrev','COREM_CrudePrev','CSMOKING_CrudePrev','DENTAL_CrudePrev','DIABETES_CrudePrev','HIGHCHOL_CrudePrev', 'KIDNEY_CrudePrev', 'LPA_CrudePrev', 'MAMMOUSE_CrudePrev', 'MHLTH_CrudePrev', 'OBESITY_CrudePrev','PHLTH_CrudePrev','SLEEP_CrudePrev','STROKE_CrudePrev','TEETHLOST_CrudePrev']].reset_index(drop=True)
health_data.head()


,Geolocation,Place_TractID,Population2010,ACCESS2_CrudePrev,ARTHRITIS_CrudePrev,BINGE_CrudePrev,BPHIGH_CrudePrev,BPMED_CrudePrev,CANCER_CrudePrev,CASTHMA_CrudePrev,CHD_CrudePrev,CHECKUP_CrudePrev,CHOLSCREEN_CrudePrev,COLON_SCREEN_CrudePrev,COPD_CrudePrev,COREM_CrudePrev,CSMOKING_CrudePrev,DENTAL_CrudePrev,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,KIDNEY_CrudePrev,LPA_CrudePrev,MAMMOUSE_CrudePrev,MHLTH_CrudePrev,OBESITY_CrudePrev,PHLTH_CrudePrev,SLEEP_CrudePrev,STROKE_CrudePrev,TEETHLOST_CrudePrev
0,POINT (-118.224698433 34.0470512474),0644000-06037206032,5275,28.0,19.9,13.0,31.7,70.0,4.5,8.5,6.9,63.8,66.0,48.6,6.5,18.2,15.6,43.6,15.5,38.3,4.1,33.3,77.6,15.0,29.5,17.3,38.9,4.0,18.8
1,POINT (-118.143832177 34.1633689905),0656000-06037462001,3974,23.3,16.7,16.3,27.1,63.7,3.6,9.4,4.8,62.2,65.7,52.5,6.0,19.4,17.8,47.6,11.6,31.8,3.2,27.0,80.2,15.7,32.0,15.3,40.2,3.2,18.1
2,POINT (-117.883112998 33.8820402343),0628000-06059011504,5473,11.7,8.3,18.6,13.6,49.5,2.0,9.2,2.1,59.3,54.5,59.5,3.4,34.6,13.3,61.3,4.6,21.1,1.6,16.1,81.0,14.1,18.7,8.5,33.4,1.2,11.9
3,POINT (-94.7523262965 32.5544549842),4843888-48183000401,3371,19.3,21.1,20.9,28.4,72.4,5.2,9.3,5.4,66.6,68.9,60.7,6.1,35.6,20.5,58.0,9.2,33.9,2.5,25.1,74.3,13.5,33.9,11.8,34.9,2.7,15.1
4,POINT (-89.3623704161 43.1211036947),5548000-55025002200,4254,10.5,23.3,25.0,27.9,74.6,6.8,9.2,6.0,70.0,76.0,70.8,5.7,29.7,16.6,72.1,8.7,34.6,2.7,18.5,80.9,11.8,29.5,11.3,30.8,2.9,12.9


In [13]:
health_data['longitude'] = np.nan
health_data['latitude'] = np.nan
i = 0
while i  < health_data.shape[0]:
  a = health_data.loc[i, 'Geolocation']
  a = a.split()
  health_data.loc[i,'longitude'] = float(a[1].replace('(', ''))
  health_data.loc[i,'latitude'] = float(a[2].replace(')', ''))
  i += 1

In [14]:
health_data

,Geolocation,Place_TractID,Population2010,ACCESS2_CrudePrev,ARTHRITIS_CrudePrev,BINGE_CrudePrev,BPHIGH_CrudePrev,BPMED_CrudePrev,CANCER_CrudePrev,CASTHMA_CrudePrev,CHD_CrudePrev,CHECKUP_CrudePrev,CHOLSCREEN_CrudePrev,COLON_SCREEN_CrudePrev,COPD_CrudePrev,COREM_CrudePrev,CSMOKING_CrudePrev,DENTAL_CrudePrev,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,KIDNEY_CrudePrev,LPA_CrudePrev,MAMMOUSE_CrudePrev,MHLTH_CrudePrev,OBESITY_CrudePrev,PHLTH_CrudePrev,SLEEP_CrudePrev,STROKE_CrudePrev,TEETHLOST_CrudePrev,longitude,latitude
0,POINT (-118.224698433 34.0470512474),0644000-06037206032,5275,28.0,19.9,13.0,31.7,70.0,4.5,8.5,6.9,63.8,66.0,48.6,6.5,18.2,15.6,43.6,15.5,38.3,4.1,33.3,77.6,15.0,29.5,17.3,38.9,4.0,18.8,-118.224698,34.047051
1,POINT (-118.143832177 34.1633689905),0656000-06037462001,3974,23.3,16.7,16.3,27.1,63.7,3.6,9.4,4.8,62.2,65.7,52.5,6.0,19.4,17.8,47.6,11.6,31.8,3.2,27.0,80.2,15.7,32.0,15.3,40.2,3.2,18.1,-118.143832,34.163369
2,POINT (-117.883112998 33.8820402343),0628000-06059011504,5473,11.7,8.3,18.6,13.6,49.5,2.0,9.2,2.1,59.3,54.5,59.5,3.4,34.6,13.3,61.3,4.6,21.1,1.6,16.1,81.0,14.1,18.7,8.5,33.4,1.2,11.9,-117.883113,33.882040
3,POINT (-94.7523262965 32.5544549842),4843888-48183000401,3371,19.3,21.1,20.9,28.4,72.4,5.2,9.3,5.4,66.6,68.9,60.7,6.1,35.6,20.5,58.0,9.2,33.9,2.5,25.1,74.3,13.5,33.9,11.8,34.9,2.7,15.1,-94.752326,32.554455
4,POINT (-89.3623704161 43.1211036947),5548000-55025002200,4254,10.5,23.3,25.0,27.9,74.6,6.8,9.2,6.0,70.0,76.0,70.8,5.7,29.7,16.6,72.1,8.7,34.6,2.7,18.5,80.9,11.8,29.5,11.3,30.8,2.9,12.9,-89.362370,43.121104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27205,POINT (-122.407558152 37.802272509),0667000-06075010400,4545,5.3,15.6,24.1,22.7,69.4,5.8,7.3,4.0,65.0,81.0,73.0,3.3,38.2,9.4,77.8,6.7,30.8,2.0,12.8,81.0,8.4,15.4,7.1,27.6,1.8,5.0,-122.407558,37.802273
27206,POINT (-89.3744412801 43.0873306533),5548000-55025001802,2784,7.3,11.5,33.7,15.2,59.9,3.4,8.3,2.5,62.9,66.8,72.7,2.8,32.5,12.2,77.6,3.9,23.3,1.5,10.9,83.8,10.3,23.4,6.4,27.4,1.2,7.6,-89.374441,43.087331
27207,POINT (-86.7590618024 36.1790862802),4752006-47037011900,2429,19.5,24.3,14.8,38.6,76.4,4.2,11.8,5.4,70.4,68.6,60.4,7.3,21.9,25.5,52.5,12.6,35.5,3.1,31.1,83.5,15.8,38.5,14.6,44.9,3.8,23.1,-86.759062,36.179086
27208,POINT (-104.784179329 39.6301998349),0804000-08005085700,4608,11.0,16.6,22.3,20.1,61.0,4.1,8.7,2.9,55.5,74.2,62.0,3.6,38.3,17.3,71.4,4.7,26.0,1.7,13.9,73.7,11.1,22.0,8.2,30.6,1.6,8.6,-104.784179,39.630200


# Get data from OPENSTREETMAP

In [33]:
import requests
import json


overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json][timeout:900];
area[name="United States"][admin_level=2]->.s;
( 
  node(area.s)["public_transport"="platform"];
)->._;
out center;
"""
response = requests.get(overpass_url,
                        params={'data': overpass_query})
data = response.json()


JSONDecodeError: ignored

In [22]:
data

{'elements': [],
 'generator': 'Overpass API 0.7.56.8 7d656e78',
 'osm3s': {'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.',
  'timestamp_areas_base': '2021-02-19T11:06:03Z',
  'timestamp_osm_base': '2021-02-19T11:49:03Z'},
 'remark': 'runtime error: Query timed out in "query" at line 5 after 181 seconds.',
 'version': 0.6}

https://janakiev.com/blog/openstreetmap-with-python-and-overpass-api/
Loading Data from OpenStreetMap with Python and the Overpass API

In [23]:
import matplotlib.pyplot as plt

# Collect coords into list
coords = []
for element in data['elements']:
  if element['type'] == 'node':
    lon = element['lon']
    lat = element['lat']
    coords.append((lon, lat))
  elif 'center' in element:
    lon = element['center']['lon']
    lat = element['center']['lat']
    coords.append((lon, lat))

# Convert coordinates into numpy array
X = np.array(coords)

plt.plot(X[:, 0], X[:, 1], 'o')
plt.title('POI in US')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.axis('equal')
plt.show()

IndexError: ignored

In [ ]:

X.shape

In [ ]:
X= pd.DataFrame(X)
X



# Turn health data and POI data into GeoDataFrame

In [ ]:

health_data_GEO = gpd.GeoDataFrame(health_data,   geometry=gpd.points_from_xy(health_data.longitude, health_data.latitude))
health_data_GEO.set_crs(epsg=4326, inplace=True)


In [ ]:
POI_GEO = gpd.GeoDataFrame(X,   geometry=gpd.points_from_xy(X[0], X[1]))
POI_GEO.set_crs(epsg=4326, inplace=True)
POI_GEO

# Setup sjoin (run twice)

In [ ]:
!apt-get install -qq curl g++ make

In [ ]:
!curl -L http://download.osgeo.org/libspatialindex/spatialindex-src-1.8.5.tar.gz | tar xz

In [ ]:
os.chdir('spatialindex-src-1.8.5')

In [ ]:
!mount -o remount,exec /content

In [ ]:
!./configure

In [ ]:
!make

In [ ]:
!make install

In [ ]:
!apt install libspatialindex-dev
!pip install rtree
!pip install pygeos

In [ ]:
!ldconfig

In [ ]:
!apt install libspatialindex-dev
!pip install rtree

In [ ]:
import rtree
import pygeos

# Space join

In [ ]:
health_join = pd.merge(area_boundary, health_data, left_on='plctract10', right_on = 'Place_TractID')
health_join_GEO = gpd.GeoDataFrame(health_join, crs="EPSG:4326", geometry='geometry_x')

In [ ]:
POI_join = gpd.sjoin(POI_GEO, health_join_GEO.to_crs(epsg=4326), how='right',op='within')
POI_join.head()

In [ ]:
POI_join.info()

In [ ]:
res = POI_join[[0, 1,'Place_TractID','area']]
res

In [ ]:
res.nunique()

# Get Count for Each GEOID


In [ ]:
count = res.groupby('Place_TractID').count()
count


In [ ]:
res2 = pd.merge(res, count, on = ['Place_TractID'], how = 'inner')
res2

In [ ]:

run_count = res2.drop_duplicates(keep = 'first', subset = 'Place_TractID')
run_count = run_count.loc[:,['0_x','1_x','Place_TractID','area_x','0_y']]
run_count.reset_index(drop=True)


# Get points distance standard deviation

In [ ]:
mean = res2.groupby('Place_TractID').mean()
mean

In [ ]:

run_std =  pd.merge(res2, mean, on = ['Place_TractID'])
run_std



In [ ]:
run_std['(x-c)^2 + (y-c)^2'] = (run_std['0_x_x'] - run_std['0_x_y'])**2 + (run_std['1_x_x'] - run_std['1_x_y'])**2
run_std = run_std.loc[:,['(x-c)^2 + (y-c)^2', 'Place_TractID']].groupby('Place_TractID').sum()
run = pd.merge(run_count, run_std, on = ['Place_TractID'], how = 'right')
run

In [ ]:
run['STD'] = np.nan

for i in range (0,run.shape[0]):
  run.at[i, 'STD'] = math.sqrt(run.at[i, '(x-c)^2 + (y-c)^2'])/run.at[i, '0_y']

run

In [ ]:
s = pd.merge(run, health_join, on = ['Place_TractID'], how = 'left')
s

In [ ]:
run = run[['STD','0_y']]
run

In [ ]:
US_heal = s[['area','Population2010','ACCESS2_CrudePrev','ARTHRITIS_CrudePrev','BINGE_CrudePrev','BPHIGH_CrudePrev','BPMED_CrudePrev','CANCER_CrudePrev','CASTHMA_CrudePrev', 'CHD_CrudePrev', 'CHECKUP_CrudePrev','CHOLSCREEN_CrudePrev','COLON_SCREEN_CrudePrev','COPD_CrudePrev','COREM_CrudePrev','CSMOKING_CrudePrev','DENTAL_CrudePrev','DIABETES_CrudePrev','HIGHCHOL_CrudePrev', 'KIDNEY_CrudePrev', 'LPA_CrudePrev', 'MAMMOUSE_CrudePrev', 'MHLTH_CrudePrev', 'OBESITY_CrudePrev','PHLTH_CrudePrev','SLEEP_CrudePrev','STROKE_CrudePrev','TEETHLOST_CrudePrev']]

In [ ]:
US_heal

In [ ]:
US_heal.info()

# Export data

In [ ]:
os.chdir("/content/gdrive/My Drive/big data and city 2/")

In [ ]:
run.to_csv('public_transport_platform.csv', header=False, index=False)
#US_heal.to_csv('US_heal.csv', header=False, index=False)